# Sztuczne sieci neuronowe: ćwiczenie 4

## 1. Opis i cel ćwiczenia
Celem ćwiczenia jest zapoznanie się z:
1. **Modelem perceptronu** i zagadnieniem klasyfikacji punktów w przestrzeni 2D z użyciem środowiska Python.
2. **Sieciami jednokierunkowymi (feed-forward)** i ich zastosowaniem do aproksymacji funkcji przy użyciu różnych algorytmów uczenia:
   - train_gd (gradient descent),
   - train_gdm (gradient descent with momentum),
   - train_gda (gradient descent with adaptive learning rate).
3. **Wpływem liczby próbek uczących** (mały vs duży zbiór) na proces uczenia i dokładność sieci.
4. **Siecią rekurencyjną Elmana** – zapoznanie się ze strukturą, działaniem i wpływem liczby neuronów w warstwie ukrytej na dokładność odtwarzania sygnału.

## 2. Wstęp teoretyczny

### 2.1. Perceptron
Perceptron jest jedną z najprostszych sieci neuronowych – zawiera warstwę wejściową (złożoną z neuronów wejściowych – nieprzetwarzających) oraz pojedynczy neuron wyjściowy z odpowiednią funkcją aktywacji (np. funkcją skoku). Perceptron dokonuje **liniowej separacji** zbioru uczącego w przestrzeni wejściowej. Jeśli punkty uczące są nieliniowo separowalne, perceptron nie jest w stanie wyuczyć się prawidłowego rozdzielenia tych punktów.

### 2.2. Sieć jednokierunkowa (feed-forward) do aproksymacji funkcji
Wielowarstwowe sieci perceptronowe (MLP – Multi-Layer Perceptron) stosowane są m.in. do aproksymacji funkcji. Typowo składają się one z:
- warstwy wejściowej,
- jednej lub więcej warstw ukrytych z nieliniową funkcją aktywacji (np. sigmoidalną),
- warstwy wyjściowej (np. z liniową funkcją aktywacji).

W niniejszym ćwiczeniu wykorzystano bibliotekę **neurolab** w języku Python, która udostępnia różne metody uczenia (m.in. `train_gd`, `train_gdm`, `train_gda`).

#### Algorytmy uczenia:
- **train_gd** – zwykła metoda spadku wzdłuż gradientu (ang. *gradient descent*).
- **train_gdm** – metoda spadku wzdłuż gradientu z momentum (ang. *gradient descent with momentum*), gdzie dodatkowo uwzględniany jest człon bezwładności (momentu).
- **train_gda** – metoda spadku wzdłuż gradientu z adaptacyjnym doborem współczynnika uczenia (ang. *gradient descent with adaptive learning rate*).


### 2.3. Sieć Elmana
Sieć Elmana jest przykładem **sieci rekurencyjnej**, w której wprowadza się sprzężenie zwrotne z warstwy ukrytej do tzw. warstwy kontekstowej. Warstwa kontekstowa przechowuje stany neuronów ukrytych z poprzedniej chwili czasowej (poprzez opóźnienie jednostkowe). Dzięki temu sieć może „pamiętać” poprzednie stany i uwzględniać je w aktualnych obliczeniach.

Strukturalnie, sygnały z warstwy ukrytej wracają do wejścia (lub do warstwy poprzedniej) przez blok opóźniający `z^-1`, co pozwala analizować ciągi czasowe czy zadania, w których potrzebna jest informacja o poprzednich stanach.

Schemat sieci Elmana można opisać następująco:
- Wejścia: \( x_0(k), x_1(k), ..., x_N(k) \),
- Kontekst (przechowuje poprzednie wyjścia warstwy ukrytej): \( v_1(k-1), ..., v_K(k-1) \),
- Warstwa ukryta o sygnałach wyjściowych \( v_i(k) \),
- Warstwa wyjściowa generująca sygnały \( y_i(k) \).

Wagi aktualizowane są metodą gradientową uwzględniającą rekurencję.

## 3. Przebieg zadania

### 3.1. Zadanie 1: „Program 1 – modelowanie działania perceptronu”

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Przykładowe punkty uczące
# Każdy wiersz to [x1, x2], label
train_data = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [1, 0, 0],
    [1, 1, 1]
    # Można dodać punkty nieliniowo separowalne
])

X = train_data[:, 0:2]
y = train_data[:, 2]

# Inicjalizacja wag
w = np.random.rand(3)  # w0 = bias, w1, w2

def perceptron_predict(x, w):
    # x = [1, x1, x2] (z dodaną jedynką na początku)
    return 1 if np.dot(w, x) >= 0 else 0

def train_perceptron(X, y, w, epochs=10, eta=0.1):
    # X – macierz Nx2, y – etykiety, w – wektor wag
    X_bias = np.c_[np.ones(X.shape[0]), X]  # dodanie kolumny jedynek
    for _ in range(epochs):
        for i in range(len(X)):
            y_hat = perceptron_predict(X_bias[i], w)
            e = y[i] - y_hat
            # aktualizacja wag
            w += eta * e * X_bias[i]
    return w

# Proces uczenia

In [ ]:
w_learned = train_perceptron(X, y, w, epochs=20, eta=0.1)

# Testowanie

In [ ]:

for i in range(len(X)):
    x_bias = np.array([1, X[i,0], X[i,1]])
    print(f"Punkt {X[i]} -> przewidywana klasa = {perceptron_predict(x_bias, w_learned)}, rzeczywista = {y[i]}")


Obserwacje:

Dla punktów liniowo separowalnych perceptron zbiega do rozwiązania, które klasyfikuje zbiór uczący poprawnie.
Dodanie punktów niespełniających kryterium liniowej separowalności powoduje, że perceptron nie jest w stanie rozdzielić wszystkich punktów poprawnie. W praktyce wagi będą się wahać i sieć nie osiągnie 100% poprawności.

## 3.2
### Zadanie 2: „Program 2. Modelowanie zagadnienia dopasowania za pomocą sieci typu feed-forward”
Cel: Zamodelować uczenie sieci MLP aproksymującej dwie funkcje (zaimplementowane w kodzie) i obliczyć błąd średniokwadratowy (MSE) dla różnych:

- Liczb neuronów w warstwie ukrytej: 3, 5, 10, 20, 30, 50
- Metod uczenia: train_gd, train_gdm, train_gda

Następnie powtórzyć eksperyment z większą liczbą próbek (np. 130 zamiast 30).

In [ ]:
import neurolab as nl
import numpy as np
import matplotlib.pyplot as plt

# Przykładowa funkcja do aproksymacji:
def f1(x):
    return np.sin(x)

def f2(x):
    return np.cos(x)

# Generowanie danych
x_min, x_max = -2*np.pi, 2*np.pi
num_samples = 30  # lub 130 w drugim podejściu
x = np.linspace(x_min, x_max, num_samples).reshape(num_samples,1)
y1 = f1(x).reshape(num_samples,1)
y2 = f2(x).reshape(num_samples,1)

# Można wybrać jedną z funkcji:
input_data = x
target_data = y1  # lub y2

# Parametry
hidden_neurons_list = [3, 5, 10, 20, 30, 50]
train_methods = ['train_gd', 'train_gdm', 'train_gda']

results = {}

for tm in train_methods:
    results[tm] = []
    for hn in hidden_neurons_list:
        # Tworzymy sieć MLP: 1 wejście, 1 warstwa ukryta (hn neuronów), 1 wyjście
        net = nl.net.newff([[x_min, x_max]], [hn, 1], [nl.trans.TanSig(), nl.trans.PureLin()])
        mse = np.mean((output - target_data)**2)
        results[tm].append(mse)

        # Ustawiamy metodę uczenia
        net.trainf = getattr(nl.train, tm)

        # Trening
        error = net.train(input_data, target_data, epochs=500, show=100, goal=0.001)

        # Symulacja
        output = net.sim(input_data)

        # Wyliczenie MSE
        mse = np.mean((output - target_data)**2)
        results[tm].append(mse)

        # Można zapisać lub wyświetlić wykres
        # plt.plot(error)
        # plt.show()

# Wyświetlenie podsumowania
for tm in train_methods:
    print(f"Metoda uczenia: {tm}")
    for hn, mse_val in zip(hidden_neurons_list, results[tm]):
        print(f"  Liczba neuronów: {hn}, MSE = {mse_val}")

In [ ]:
from IPython.display import display, Latex
# Budujemy tabelę w LaTeX
table_lines = []
table_lines.append(r"\begin{table}[h]")
table_lines.append(r"\centering")
table_lines.append(r"\begin{tabular}{l|cccccc}")
table_lines.append(r"\hline")
table_lines.append(r"\textbf{Liczba neuronów} & " + " & ".join(str(hn) for hn in hidden_neurons_list) + r" \\")
table_lines.append(r"\hline")

for tm in train_methods:
    # np. "MSE (train_gd)"
    row_label = f"MSE ({tm})"
    # Formatowanie liczb (np. 5 miejsc po przecinku)
    row_values = [f"{val:.5f}" for val in results[tm]]
    row_str = row_label + " & " + " & ".join(row_values) + r" \\"
    table_lines.append(row_str)

table_lines.append(r"\hline")
table_lines.append(r"\end{tabular}")
table_lines.append(r"\caption{Porównanie wartości MSE dla różnych metod uczenia i liczby neuronów}")
table_lines.append(r"\label{tab:mse_auto}")
table_lines.append(r"\end{table}")

# Łączymy linie w jeden string
table_code = "\n".join(table_lines)

display(Latex(table_code))

## 3.3. Sieci rekurencyjne na przykładzie sieci Elmana (Zadanie 3)
W zadaniu wykorzystujemy przykładowy Program 3 (poniżej fragment kodu), w którym testujemy zdolność sieci Elmana do odtwarzania sygnału prostokątnego o wartościach [1, 2]. Sprawdzamy wpływ liczby neuronów w warstwie ukrytej na dokładność odwzorowania.

In [ ]:
import neurolab as nl
import numpy as np
import pylab as pl

# Create train samples
i1 = np.sin(np.arange(0, 20))
i2 = np.sin(np.arange(0, 20)) * 2
t1 = np.ones([1, 20])
t2 = np.ones([1, 20]) * 2

input = np.array([i1, i2, i1, i2]).reshape(20 * 4, 1)
target = np.array([t1, t2, t1, t2]).reshape(20 * 4, 1)

# Create network with 2 layers
# newelm - tworzy sieć Elmana
net = nl.net.newelm([[-2, 2]], [20, 1], [nl.trans.TanSig(), nl.trans.PureLin()])

# Inicjalizacja
net.layers[0].initf = nl.init.InitRand([-0.1, 0.1], 'wb')
net.layers[1].initf = nl.init.InitRand([-0.1, 0.1], 'wb')
net.init()

# Trening
error = net.train(input, target, epochs=500, show=100, goal=0.01)

# Symulacja
output = net.sim(input)

# Rysunki
pl.subplot(211)
pl.plot(error)
pl.xlabel('Epoch number')
pl.ylabel('Train error (default MSE)')
pl.subplot(212)
pl.plot(target.reshape(80))
pl.plot(output.reshape(80))
pl.legend(['train target', 'net output'])
pl.show()

In [ ]:
# Lista różnych rozmiarów warstwy ukrytej
hidden_neurons_list = [3, 5, 20, 50, 100]

# Tablica do przechowywania wyników (MSE)
mse_results = []

# Pętla po różnych liczbach neuronów
for hn in hidden_neurons_list:
    # Tworzymy sieć Elmana (newelm)
    # Zakładamy wejście np. w przedziale [-2,2]; dostosuj w razie potrzeby
    net = nl.net.newelm([[-2, 2]], [hn, 1], [nl.trans.TanSig(), nl.trans.PureLin()])

    # Inicjalizacja wag
    net.init()

    # Trening (np. 500 epok, goal=0.01)
    # W praktyce parametry ustalasz w zależności od zadania
    error = net.train(input_data, target_data, epochs=500, show=100, goal=0.01)

    # Symulacja
    output = net.sim(input_data)

    # Obliczenie błędu średniokwadratowego
    mse = np.mean((output - target_data) ** 2)

    # Dodaj wynik do listy
    mse_results.append(mse)

# --- Generowanie tabeli w LaTeX ---
print(r"\begin{table}[h]")
print(r"\centering")
# Liczba kolumn = 1 (opis) + tyle kolumn, ile rozmiarów warstwy ukrytej
print(r"\begin{tabular}{l|" + "c"*len(hidden_neurons_list) + "}")
print(r"\hline")

# Nagłówek: "Liczba neuronów w warstwie ukrytej"
print("Liczba neuronów w warstwie ukrytej & " + " & ".join(str(hn) for hn in hidden_neurons_list) + r" \\")
print(r"\hline")

# Drugi wiersz: "Średniokwadratowy błąd"
# Możemy sformatować liczby, np. do 4 miejsc po przecinku
mse_str = " & ".join(f"{m:.4f}" for m in mse_results)
print("Średniokwadratowy błąd & " + mse_str + r" \\")

print(r"\hline")
print(r"\end{tabular}")
print(r"\caption{Wpływ liczby neuronów w warstwie ukrytej na MSE (sieć Elmana)}")
print(r"\label{tab:elman_mse}")
print(r"\end{table}")

### 3.3.2 Obserwacje
Wraz ze wzrostem liczby neuronów w warstwie ukrytej sieć Elmana zwykle lepiej (z mniejszym błędem) odwzorowuje sygnał, jednak nadmierne zwiększanie liczby neuronów może prowadzić do wydłużenia czasu treningu lub przeuczenia w innych zastosowaniach.
Dla prostego sygnału prostokątnego [1, 2] już niewielka liczba neuronów (np. 5–20) może zapewnić zadowalającą dokładność.
Optymalny rozmiar warstwy ukrytej zależy od charakteru sygnału i liczby dostępnych próbek.


# 4. Wnioski końcowe
Perceptron skutecznie rozdziela dane, o ile są liniowo separowalne. Przy nieliniowo separowalnych nie osiąga zadowalających wyników – należy wtedy stosować bardziej złożone sieci.
Sieci MLP (feed-forward) z jedną warstwą ukrytą mogą aproksymować różne funkcje, a jakość aproksymacji zależy od:
Liczby neuronów w warstwie ukrytej (zbyt mała – niedouczenie, zbyt duża – możliwe przeuczenie).
Metody uczenia (train_gd, train_gdm, train_gda) – dodanie momentu lub adaptacyjnego współczynnika uczenia może przyspieszyć i ustabilizować proces uczenia.
Wielkości zbioru uczącego – większy zbiór zwykle poprawia uogólnianie.
Sieć Elmana dzięki rekurencji potrafi „pamiętać” poprzednie stany i lepiej odwzorowywać sygnały czasowe (np. sygnał prostokątny). Liczba neuronów w warstwie ukrytej znacząco wpływa na dokładność i stabilność uczenia.
Istnieje kompromis pomiędzy złożonością sieci (liczbą neuronów) a ryzykiem przeuczenia i czasem uczenia. W praktyce liczba neuronów w warstwie ukrytej dobierana jest eksperymentalnie, w oparciu o wiedzę o zadaniu i testy empiryczne.
